#WHISKY IS GOD

##This program is recommending new whisky based on your favourite one.

***The database consists of 109 single malts and 5 features which describe them.***

----------------------------------------------------------------------

Thomas fav one is: ***Lagavulin***; therefore using K-nearest Neighbours Classifier the best recommendation will be: Laphroaig

----------------------------------------------------------------------


In [3]:
# set up env
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [15]:
# import data:
dir = 'C:\\Users\\Lenovo\\PycharmProjects\\RandomPythonProjects\\Project10_Whisky\\ScotchData\\Scotch data (Windows)\\'
file = 'scotch.xlsx'
data = pd.read_excel(dir + file, sheet_name='scotch.xls', header=0)

In [172]:
# whisky db
print('_'*90, '\n\nAVAILABLE WHISKY IN DATA SET\n', '_'*90, '\n')
for letter in data.full_name.str[0].unique():
        print(letter, ':\t', [element for element in list(data.full_name) if element.startswith(letter)])
print('_'*90)

__________________________________________________________________________________________ 

AVAILABLE WHISKY IN DATA SET
 __________________________________________________________________________________________ 

A :	 ['Aberfeldy', 'Aberlour', 'Ardberg', 'Ardmore', 'Auchentoshan', 'Aultmore']
B :	 ['Balblair', 'Balmenach', 'Balvenie', 'Banff', 'Ben Nevis', 'Benriach', 'Benrinnes', 'Benromach', 'Bladnoch', 'Blair Athol', 'Bowmore', 'Brackla', 'Bruichladdich', 'Bunnahabhain']
C :	 ['Caol Ila', 'Caperdonich', 'Cardhu', 'Clynelish', 'Coleburn', 'Convalmore', 'Cragganmore', 'Craigellachie']
D :	 ['Dailuaine', 'Dallas Dhu', 'Dalmore', 'Dalwhinnie', 'Deanston', 'Dufftown']
E :	 ['Edradour']
F :	 ['Fettercairn']
G :	 ['Glen  Albyn', 'Glenallachie', 'Glenburgie', 'Glencadam', 'Glen Deveron', 'Glendronach', 'Glendullan', 'Glen Elgin', 'Glenesk', 'Glenfarclas', 'Glenfiddich', 'Glen Garioch', 'Glenglassaugh', 'Glengoyne', 'Glen Grant', 'Glen Keith', 'Glenkinchie', 'Glenlivet', 'Glenlochy', 'Gle

In [169]:
data['id'] = data.index + 1
knn_columns = ['full_name', 'short_name', '_AGE', '_DIST', '_SCORE', '_%', '_REGION', '_DISTRICT',
       '_islay', '_midland', '_spey', '_east', '_west', '_north ', '_lowland',
       '_campbell', '_islands']
X_train = data.drop(knn_columns, axis=1)
y_train = data['id']
X_test = data.query('full_name == "Lagavulin"').drop(knn_columns, axis=1)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
distance, indeces = knn.kneighbors(X_test, 6)
print('_'*70, '\nTaking into cosideration 5 distinct Whisky characteristics,',
              'Most recommended scotch (based on Lagavulin) are:', '_'*70, sep='\n')
pd.concat([data.iloc[indeces[0], [0, -16, -12]], 
           pd.Series(distance[0], index=[77, 78, 76, 79, 75, 80], name='Closest')], axis=1).iloc[1:, :]


______________________________________________________________________

Taking into cosideration 5 distinct Whisky characteristics,
Most recommended scotch (based on Lagavulin) are:
______________________________________________________________________


,full_name,_AGE,_REGION,Closest
78,Laphroaig,10.0,ISLAY,3.872983
76,Ladyburn,20.0,LOW,4.472136
79,Linkwood,12.0,HIGH,4.690416
75,Knockdhu,-9.0,HIGH,4.795832
80,Littlemill,8.0,LOW,5.477226
